# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [41]:
# imports
import os
from pathlib import Path
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import json
from textwrap import indent
import nbformat


In [38]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'gemma3'
api = '123...'

In [42]:
# set up environment
load_dotenv(override=True)

True

In [43]:
system_prompt = """
You are an expert code reviewer. 
you are provided with the python or python notebook files. 
you will review the code and suggest the areas where the code can be made more efficient.
you will also review the industry best practises for the code suggest any new edits to be incorporated for the efficient code 
you will also check for any comments and suggest if there is any improvement. check for any hardcoded values like passwords, or api keys and flag it
only the files with extension .py or .ipynb will be considered ignore the rest.
you will respond the output in the Markdown format for example: 
## test.py
- please explain what this function def func does
- Please dont hard code any passwords
"""

In [44]:

def extract_code_fron_ipynb(filePath):
    with open(filePath,'r',encoding="utf-8") as f:
        nb = nbformat.read(f, as_version=4)
    
    return "\n\n".join([c.source for c in nb.cells if c.cell_type=='code'])

def get_file_content(filePath):
    suffix = Path(filePath).suffix

    if suffix == '.py':
        return Path(filePath).read_text
    elif suffix == '.ipynb':
        return extract_code_fron_ipynb(filePath)

def get_list_files(filePath):
    path = Path(filePath)
    files = {
        f.name: get_file_content(str(path / f.name)) for f in path.iterdir() if f.suffix in ['.py','.ipynb']
    }
    return files
    

In [45]:
filePath = '/Users/mia/myprojects/uv_projects/llm_engineering/week1'
filesList = get_list_files(filePath)

In [46]:
# here is the question; type over this to ask something new

question = f"""
Please explain what this code does and why:
the name and code is listed as below json file 
{filesList}
"""

question

'\nPlease explain what this code does and why:\nthe name and code is listed as below json file \n{\'scraper.py\': <bound method Path.read_text of PosixPath(\'/Users/mia/myprojects/uv_projects/llm_engineering/week1/scraper.py\')>, \'solution.py\': <bound method Path.read_text of PosixPath(\'/Users/mia/myprojects/uv_projects/llm_engineering/week1/solution.py\')>, \'test.py\': <bound method Path.read_text of PosixPath(\'/Users/mia/myprojects/uv_projects/llm_engineering/week1/test.py\')>, \'week1 EXERCISE.ipynb\': \'# imports\\nimport os\\nfrom pathlib import Path\\nfrom dotenv import load_dotenv\\nfrom IPython.display import Markdown, display, update_display\\nfrom openai import OpenAI\\nimport json\\n\\n# constants\\n\\nMODEL_GPT = \\\'gpt-4o-mini\\\'\\nMODEL_LLAMA = \\\'gemma3\\\'\\napi = \\\'123...\\n\\n# set up environment\\nload_dotenv(override=True)\\n\\nsystem_prompt = """\\nYou are an expert code reviewer. \\nyou are provided with the python or python notebook files. \\nyou will r

In [47]:
messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ]

In [ ]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()
stream = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=messages,
    stream=True
)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    update_display(Markdown(response), display_id=display_handle.display_id)

## scraper.py
- The function `get_file_content` is missing a call to `read_text()`; it should be changed to `return Path(filePath).read_text()` with parentheses to actually read the file.
- Consider handling exceptions for file operations to avoid crashing if a file can't be read or doesn't exist.

## solution.py
- Ensure that no passwords or API keys are hardcoded. You should load sensitive information from environment variables.
- Add docstrings to your functions to clarify their purpose and usage.

## test.py
- Please elaborate on the purpose of the function definitions; it's important to provide context in your comments.
- Hardcoded values such as passwords or API keys should be avoided. Implement handling through environment files instead.
- Review the overall structure for performance optimization, e.g., reducing repeated calls for similar operations.

## week1 EXERCISE.ipynb
- There's an API key hardcoded as `api = '123...'`; consider removing it and loading sensitive information from environment variables.
- To improve readability, consider using consistent naming conventions and meaningful variable names.
- Use comments and markdown cells to better document your code logic and transitions between sections.

## day1.ipynb
- You have hardcoded API keys in multiple locations; make sure to handle them via a `.env` file and load them using `load_dotenv`.
- Documentation can be improved by adding markdown explanations for logic and steps before code blocks.

## day2.ipynb
- There is a hardcoded Google API key. API keys should be loaded from environment variables instead.
- It’s good practice to check the response code when making API calls to handle possible errors effectively.

## day4.ipynb
- The API key is loaded from the environment, which is a good practice. Ensure that the `.env` file is excluded from version control (e.g., using `.gitignore`).
- Ensure consistency in error messages and validations regarding API key settings.

## day5.ipynb
- You're exposing API keys in your code. Ensure they're never hardcoded; use environment variables instead.
- It’s beneficial to include type hints in functions for clearer type expectations.

In general, ensure secure handling of sensitive information, improve code readability through comments and documentation, and adopt consistent naming and structure across files for better maintainability.

In [50]:
# Get Llama 3.2 to answer

llama=OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
llama_stream = llama.chat.completions.create(
    model=MODEL_LLAMA,
    messages=messages
)
response = llama_stream.choices[0].message.content
print(response)

Okay, I've reviewed the Jupyter Notebook code you provided. Here's a breakdown of its functionality, potential issues, and suggestions for improvement:

**Overall Functionality**

The notebook is designed to:

1. **Scrape a Website:** Uses the `scraper.py` (which you haven't provided, but assumed to exist) to fetch the HTML content of a website (initially `https://edwarddonner.com`).
2. **Extract Relevant Links:**  Processes the scraped content to identify key links that could be included in a brochure (focused on company information)
3. **Generate a Brochure:** Constructs a brochure summary from these links -  likely using a GPT model (`gpt-5-nano`).

**Code Breakdown**

*   **Import Statements:** Imports necessary libraries like `os`, `json`, `dotenv`, `IPython.display` (for Markdown rendering), and `scraper` and `openai`.
*   **API Key Handling:** Attempts to load the OpenAI API key from the environment variable `OPENAI_API_KEY`. Includes basic error handling if the key isn't found 